In [1]:
import pandas as pd

from data_loading import (load_valid_isoform_clones,
    load_isoform_and_paralog_y2h_data, 
    load_y1h_pdi_data, 
    load_m1h_activation_data)


In [2]:
df = pd.read_excel('../data/internal/from_kaia/prioritized_BRCA_isos_to_clone.CDS_seqs.xlsx')
iso = load_valid_isoform_clones()
iso['cds_len'] = iso['cds'].str.len()

y2h = load_isoform_and_paralog_y2h_data()
y1h = load_y1h_pdi_data()
m1h = load_m1h_activation_data()

ppi_per_clone = (y2h.loc[(y2h['category'] == 'tf_isoform_ppis') &
         y2h['Y2H_result'].notnull()]
    .groupby(['ad_gene_symbol', 'ad_clone_acc'])
    .size())
clones_gte_2ppi = ppi_per_clone[ppi_per_clone >= 2].groupby('ad_gene_symbol').size()
tf_with_ppi_data = set(clones_gte_2ppi[clones_gte_2ppi >= 2].index)
df['at_least_2_ppis_for_two_isoforms'] = df['gene'].isin(tf_with_ppi_data)

y1h['any_pdi'] = y1h.iloc[:, 2:].any(axis=1)
tf_with_pdi_data = (y1h.groupby('tf').size() >= 2) & y1h.groupby('tf')['any_pdi'].any()
tf_with_pdi_data = set(tf_with_pdi_data[tf_with_pdi_data].index)
df['at_least_1_pdi'] = df['gene'].isin(tf_with_pdi_data)


df['at_least_2_m1h_data'] = df['gene'].isin(set(m1h.groupby('gene').size()[m1h.groupby('gene').size() >= 2].index))

iso['cds'].apply(lambda x: x[-3:] in {'TAG', 'TAA', 'TGA'}).value_counts()

def remove_stop_codon(s):
    if s[-3:] in {'TAG', 'TAA', 'TGA'}:
        return s[:-3]
    else:
        return s


iso['cds'] = iso['cds'].apply(remove_stop_codon)
df['seq_cds'] = df['seq_cds'].apply(remove_stop_codon)

In [3]:
# is the matching on CDS working?
df.loc[~df['seq_cds'].str.startswith('ATG'), :]

index    gene ctrl_id       primary_analysis  \
4      DMTF1:::GENCPID72720__NA__NA   DMTF1     NaN                control   
9      DMTF1:::GENCPID67064__NA__NA   DMTF1     NaN                control   
19       ESR1:::GENCPID1248__NA__NA    ESR1     NaN                control   
48        WT1:::GENCPID1670__NA__NA     WT1     NaN                control   
54        WT1:::GENCPID1669__NA__NA     WT1     NaN                control   
...                             ...     ...     ...                    ...   
2232   ADNP2:::GENCPID70189__NA__NA   ADNP2     NaN  HER2-enriched (no DE)   
2248  HMG20A:::GENCPID68556__NA__NA  HMG20A     NaN    Normal-like (no DE)   
2250    GTF3A:::GENCPID1685__NA__NA   GTF3A     NaN    Normal-like (no DE)   
2269   TCF7L2:::GENCPID1349__NA__NA  TCF7L2     NaN      bulk BRCA (no DE)   
2298   ZNF692:::NA__PACBIOPID97__NA  ZNF692     NaN      bulk BRCA (no DE)   

      analysis_effect_size  analysis_padj pick_status iso_details  \
4                      NaN            NaN     control       other   
9                      NaN            NaN     control       other   
19                     NaN            NaN     control        none   
48                     NaN            NaN     control   principal   
54                     NaN            NaN     control        none   
...                    ...            ...         ...         ...   
2232             -0.029588       0.116439       other       other   
2248              0.113382       0.000037       other       other   
2250              0.000819       0.993808       other   principal   
2269              0.000456       0.446407       other       other   
2298              0.036696       0.004655     key iso        none   

     cloneable_cell_lines best_cell_line  tpm_in_best_cell_line  \
4                    MCF7           MCF7               1.545120   
9            ZC, MDA, HCC            HCC               1.887490   
19                   none            HME               0.000000   
48                   none             ZC               0.646284   
54                   none             ZC               0.629601   
...                   ...            ...                    ...   
2232             MCF7, ZC             ZC               3.019590   
2248                 MCF7           MCF7               1.192980   
2250             MDA, HCC            MDA              25.458000   
2269                 MCF7           MCF7               1.064790   
2298                  HME            HME               3.424080   

      ratio_in_best_cell_line  appris_status  \
4                        0.07            NaN   
9                        0.13            NaN   
19                        NaN            NaN   
48                       0.21    PRINCIPAL:5   
54                       0.21  ALTERNATIVE:2   
...                       ...            ...   
2232                     0.26            NaN   
2248                     0.11            NaN   
2250                     0.19    PRINCIPAL:1   
2269                     0.10            NaN   
2298                     0.15            NaN   

                                        removal_flag   all_sig_analyses  \
4                                           no flags            control   
9                                           no flags            control   
19                                          no flags            control   
48                                          no flags            control   
54                                          no flags            control   
...                                              ...                ...   
2232  flag to remove: not in Sachi's high-conf. list               none   
2248  flag to remove: not in Sachi's high-conf. list               none   
2250                                        no flags               none   
2269                                        no flags               none   
2298                                        no flags  bul

In [4]:
iso.loc[~iso['cds'].str.startswith('ATG'), :]

gene        clone_acc  \
57    EBF1   EBF1|2/3|06D12   
58    EBF1   EBF1|3/3|06B12   
478  TEAD3  TEAD3|1/2|10E03   
479  TEAD3  TEAD3|2/2|10G04   

                                                   cds  \
57   AGGTTCTTCTTGAAATTTTTCCTCAAATGTAACCAAAATTGCCTAA...   
58   AGGTTCTTCTTGAAATTTTTCCTCAAATGTAACCAAAATTGCCTAA...   
478  ATAGCGTCCAACAGCTGGAACGCCAGCAGCAGCCCCGGGGAGGCCC...   
479  ATAGCGTCCAACAGCTGGAACGCCAGCAGCAGCCCCGGGGAGGCCC...   

                                                aa_seq  num_aa  \
57   RFFLKFFLKCNQNCLKNAGNPRDMRRFQVVVSTTVNVDGHVLAVSD...     408   
58   RFFLKFFLKCNQNCLKNAGNPRDMRRFQVVVSTTVNVDGHVLAVSD...     407   
478  IASNSWNASSSPGEAREDGPEGLDKGLDNDAEGVWSPDIEQSFQEA...     435   
479  IASNSWNASSSPGEAREDGPEGLDKGLDNDAEGVWSPDIEQSFQEA...     435   

     is_novel_isoform clone_name  cds_len  
57               True     EBF1-2     1224  
58              False     EBF1-3     1221  
478             False    TEAD3-1     1305  
479             False    TEAD3-2     1305

In [5]:
# use this. Add tf gene level PPI + PDI + M1H data
df.loc[df['index'].apply(lambda x: x.split('_')[-1].startswith('TF'))
        & df['at_least_2_ppis_for_two_isoforms']
        & df['at_least_1_pdi']
        & df['at_least_2_m1h_data'], :]

index    gene ctrl_id  \
53                  WT1:::GENCPID22245__NA__TF1P0PID361     WT1     NaN   
68      MAX:::GENCPID53245__PACBIOPID40380__TF1P0PID889     MAX     NaN   
70      MAX:::GENCPID53242__PACBIOPID40378__TF1P0PID886     MAX     NaN   
71      MAX:::GENCPID53243__PACBIOPID40379__TF1P0PID888     MAX     NaN   
74                            MAX:::NA__NA__TF1P0PID887     MAX     NaN   
76                  MAX:::GENCPID53247__NA__TF1P0PID890     MAX     NaN   
143   TEAD2:::GENCPID28155__PACBIOPID20724__TF1P0PID452   TEAD2     NaN   
151                           MLX:::NA__NA__TF1P0PID996     MLX     NaN   
156     MLX:::GENCPID60033__PACBIOPID45359__TF1P0PID995     MLX     NaN   
157                           MLX:::NA__NA__TF1P0PID994     MLX     NaN   
486    STAT3:::GENCPID11404__PACBIOPID7724__TF1P0PID151   STAT3     NaN   
1230  PPARG:::GENCPID60970__PACBIOPID46081__TF1P0PID...   PPARG     NaN   
1231              PPARG:::GENCPID28214__NA__TF1P0PID454   PPARG     NaN   
2004             ZBTB18:::GENCPID15640__NA__TF1P0PID212  ZBTB18     NaN   
2005                       ZBTB18:::NA__NA__TF1P0PID335  ZBTB18     NaN   
2244              NFIX:::GENCPID66812__NA__TF1P0PID1089    NFIX     NaN   
2246               NFIX:::GENCPID16587__NA__TF1P0PID232    NFIX     NaN   

            primary_analysis  analysis_effect_size  analysis_padj pick_status  \
53                   control                   NaN            NaN     control   
68     survival (w/ cell DE)             -1.621830   5.370295e-01       other   
70     survival (w/ cell DE)             -0.929807   7.950435e-01       other   
71     survival (w/ cell DE)              1.767979   5.187464e-01       other   
74     survival (w/ cell DE)              0.903486   8.099524e-01       other   
76     survival (w/ cell DE)              1.115621   7.344800e-01       other   
143    survival (w/ cell DE)              3.012326   1.306110e-01       other   
151    survival (w/ cell DE)              3.470959   9.233241e-02     key iso   
156    survival (w/ cell DE)              1.796551   4.946944e-01       other   
157    survival (w/ cell DE)              1.056256   7.419900e-01       other   
486   Luminal A (w/ cell DE)             -0.000502   1.166169e-01       other   
1230       Luminal A (no DE)              0.097224   9.307895e-06     key iso   
1231       Luminal A (no DE)             -0.073767   1.095464e-13       other   
2004       Luminal B (no DE)             -0.054820   1.310117e-02       other   
2005       Luminal B (no DE)             -0.001300   1.000000e+00       other   
2244     Normal-like (no DE)             -0.026534   3.679245e-01       other   
2246     Normal-like (no DE)             -0.070042   1.154686e-01       other   

                   iso_details     cloneable_cell_lines best_cell_line  \
53               brca-dominant                      MDA            MDA   
68    principal, brca-dominant  HME, MCF7, ZC, MDA, HCC            MDA   
70                       other  HME, MCF7, ZC, MDA, HCC             ZC   
71                       other                     none            HME   
74                       other                       ZC             ZC   
76                       other            HME, MDA, HCC            HCC   
143   principal, brca-dominant      HME, MCF7, MDA, HCC            MDA   
151                       none                       ZC             ZC   
156                      other                     none             ZC   
157                      other                       ZC             ZC   
486                      other                      HME            HME   
1230             brca-dominant      HME, MCF7, MDA, HCC            MDA   
1231                 principal                     none            MDA   
2004                 principal             ZC, MDA, HCC            MDA   
2005                     other             ZC, MDA, HCC             ZC   
2244                     other  HME, MCF7, ZC, MDA, HCC 

In [6]:
# WT1:::GENCPID22245__NA__TF1P0PID361 not matching CDS length in Kaia's table: 867, in isoform table: 864
a = iso.loc[iso['clone_acc'] == 'WT1|6/6|10G06']['cds'].values[0]
b = df.loc[df['index'] == 'WT1:::GENCPID22245__NA__TF1P0PID361', 'seq_cds'].values[0]
a

'ATGGAGAAGGGTTACAGCACGGTCACCTTCGACGGGACGCCCAGCTACGGTCACACGCCCTCGCACCATGCGGCGCAGTTCCCCAACCACTCATTCAAGCATGAGGATCCCATGGGCCAGCAGGGCTCGCTGGGTGAGCAGCAGTACTCGGTGCCGCCCCCGGTCTATGGCTGCCACACCCCCACCGACAGCTGCACCGGCAGCCAGGCTTTGCTGCTGAGGACGCCCTACAGCAGTGACAATTTATACCAAATGACATCCCAGCTTGAATGCATGACCTGGAATCAGATGAACTTAGGAGCCACCTTAAAGGGCCACAGCACAGGGTACGAGAGCGATAACCACACAACGCCCATCCTCTGCGGAGCCCAATACAGAATACACACGCACGGTGTCTTCAGAGGCATTCAGGATGTGCGGCGTGTGCCTGGAGTAGCCCCGACTCTTGTACGGTCGGCATCTGAGACCAGTGAGAAACGCCCCTTCATGTGTGCTTACCCAGGCTGCAATAAGAGATATTTTAAGCTGTCCCACTTACAGATGCACAGCAGGAAGCACACTGGTGAGAAACCATACCAGTGTGACTTCAAGGACTGTGAACGAAGGTTTTCTCGTTCAGACCAGCTCAAAAGACACCAAAGGAGACATACAGGTGTGAAACCATTCCAGTGTAAAACTTGTCAGCGAAAGTTCTCCCGGTCCGACCACCTGAAGACCCACACCAGGACTCATACAGGTAAAACAAGTGAAAAGCCCTTCAGCTGTCGGTGGCCAAGTTGTCAGAAAAAGTTTGCCCGGTCAGATGAATTAGTCCGCCATCACAACATGCATCAGAGAAACATGACCAAACTCCAGCTGGCGCTT'

In [7]:
iso.loc[iso['gene'] == 'NFIX']

gene       clone_acc                                                cds  \
283  NFIX  NFIX|1/4|08H03  ATGTACTCCCCGTACTGCCTCACCCAGGATGAGTTCCACCCGTTCA...   
284  NFIX  NFIX|2/4|08G06  ATGGATGAGTTCCACCCGTTCATCGAGGCACTGCTGCCTCACGTCC...   
285  NFIX  NFIX|3/4|08F05  ATGTACTCCCCGTACTGCCTCACCCAGGATGAGTTCCACCCGTTCA...   
286  NFIX  NFIX|4/4|08D05  ATGTACTCCCCGTACTGCCTCACCCAGGATGAGTTCCACCCGTTCA...   

                                                aa_seq  num_aa  \
283  MYSPYCLTQDEFHPFIEALLPHVRAFSYTWFNLQARKRKYFKKHEK...     441   
284  MDEFHPFIEALLPHVRAFSYTWFNLQARKRKYFKKHEKRMSKDEER...     433   
285  MYSPYCLTQDEFHPFIEALLPHVRAFSYTWFNLQARKRKYFKKHEK...     400   
286  MYSPYCLTQDEFHPFIEALLPHVRAFSYTWFNLQARKRKYFKKHEK...     353   

     is_novel_isoform clone_name  cds_len  
283             False     NFIX-1     1358  
284             False     NFIX-2     1334  
285              True     NFIX-3     1235  
286              True     NFIX-4     1059

## candidate genes for cancer vinyette

- MAX (lots of isoforms)
- MLX (lots of isoforms)
- NFIX?
- PPARG-1
    - data looks good...
- TEAD2-1
    - data looks good 
- WT1-6
    - we have a lot of data (already known splicing effects in cancer)
- ZBTB18
    - data looks good

In [8]:
df['seq_cds'].apply(lambda x: x[-3:] in {'TAG', 'TAA', 'TGA'}).value_counts()

False    2328
True        1
Name: seq_cds, dtype: int64

In [9]:
pd.merge(df, iso, left_on='seq_cds', right_on='cds')

index  gene_x      ctrl_id  \
0                         DMTF1:::NA__NA__TF1P0PID985   DMTF1          NaN   
1    KLF6:::GENCPID19594__PACBIOPID14061__TF1P0PID303    KLF6  KLF6:::KLF6   
2                          TP53:::NA__NA__TF1P0PID219    TP53          NaN   
3                          TP53:::NA__NA__TF1P0PID334    TP53          NaN   
4     MAX:::GENCPID53245__PACBIOPID40380__TF1P0PID889     MAX          NaN   
..                                                ...     ...          ...   
133            ZNF511:::GENCPID48453__NA__TF1P0PID827  ZNF511          NaN   
134            PLAGL1:::GENCPID13783__NA__TF1P0PID179  PLAGL1          NaN   
135                      ZNF692:::NA__NA__TF1P0PID175  ZNF692          NaN   
136                      ZNF692:::NA__NA__TF1P0PID174  ZNF692          NaN   
137  ZNF3:::GENCPID37098__PACBIOPID27951__TF1P0PID563    ZNF3          NaN   

          primary_analysis  analysis_effect_size  analysis_padj pick_status  \
0                  control                   NaN            NaN     control   
1                  control                   NaN            NaN     control   
2                  control                   NaN            NaN     control   
3                  control                   NaN            NaN     control   
4    survival (w/ cell DE)             -1.621830   5.370295e-01       other   
..                     ...                   ...            ...         ...   
133      bulk BRCA (no DE)             -0.000973   7.807209e-01       other   
134      bulk BRCA (no DE)             -0.020090   1.592444e-01       other   
135      bulk BRCA (no DE)             -0.000698   8.743089e-01       other   
136      bulk BRCA (no DE)              0.004773   6.312533e-01       other   
137      bulk BRCA (no DE)              0.045880   1.781140e-07       other   

                               iso_details     cloneable_cell_lines  \
0                                    other                MCF7, MDA   
1    principal, brca-dominant, control iso  HME, MCF7, ZC, MDA, HCC   
2                                    other  HME, MCF7, ZC, MDA, HCC   
3                                    other  HME, MCF7, ZC, MDA, HCC   
4                 principal, brca-dominant  HME, MCF7, ZC, MDA, HCC   
..                                     ...                      ...   
133                                  other                     none   
134               principal, brca-dominant                      HME   
135                                  other                     none   
136                                  other                      HME   
137                                  other  HME, MCF7, ZC, MDA, HCC   

    best_cell_line  tpm_in_best_cell_line  ratio_in_best_cell_line  \
0             MCF7               1.437810                     0.06   
1              MDA              96.764500                     0.66   
2              MDA              22.154800                     0.17   
3              MDA              16.583800                     0.13   
4              MDA              19.774901                     0.48   
..             ...                    ...                      ...   
133             ZC               3.601710                     0.04   
134            HME               2.661955                     0.56   
135             ZC               1.513540                     0.03   
136            HME               2.244680                     0.10   
137            MDA               1.301601                     0.17   

     appris_status removal_flag  \
0              NaN     no flags   
1      PRINCIPAL:1     no flags   
2              NaN     no flags   
3              NaN     no flags   
4      PRINCIPAL:4     no flags   
..             ...          ...   
133  ALTERNATIVE:2     no flags   
134    PRINCIPAL:1     no flags   
135            NaN     no flags   
136            NaN     no flags   
137  ALTERNATIVE:2     no flags   

                                      all_sig_a

In [10]:
iso['gene'].nunique()

310

In [11]:
df = pd.read_csv('../data/external/cancer_gene_census.csv')
cgc = set(df['Gene Symbol'].unique())
cgc.intersection(set(iso['gene'].unique()))

{'CREB1',
 'CTCF',
 'DDIT3',
 'EBF1',
 'ELK4',
 'ETV4',
 'ETV6',
 'FLI1',
 'FOXO3',
 'GATA1',
 'GATA2',
 'GATA3',
 'GLI1',
 'HEY1',
 'HIF1A',
 'HMGA1',
 'HOXA9',
 'KLF4',
 'KLF6',
 'LEF1',
 'MAX',
 'MITF',
 'NFE2L2',
 'PATZ1',
 'PAX5',
 'PAX7',
 'PAX8',
 'PBX1',
 'POU5F1',
 'PPARG',
 'PRDM16',
 'PRRX1',
 'RARA',
 'REL',
 'RUNX1',
 'SIX1',
 'SMAD3',
 'SMAD4',
 'STAT3',
 'TCF12',
 'TCF7L2',
 'TP53',
 'TP63',
 'WT1',
 'ZBTB16'}

In [12]:
df = df.loc[df['Gene Symbol'].isin(iso['gene'].unique()),
            ['Gene Symbol',
            'Tumour Types(Somatic)',
            'Tissue Type',
            'Molecular Genetics', 'Role in Cancer', 'Mutation Types']]

In [13]:
df.shape

(45, 6)

In [14]:
df['n_cloned_isoforms'] = df['Gene Symbol'].map(iso.groupby('gene').size())

In [15]:
df['has_novel_isoform'] = df['Gene Symbol'].map(iso.groupby('gene')['is_novel_isoform'].any())

In [16]:
(df['n_cloned_isoforms'] >= 2).sum()

36